In [1]:
import pandas as pd
import numpy as np 

pd.set_option('display.max_columns', 300)

## Read in hold out data, scalers, and best model

In [2]:
df = pd.read_csv('data/kc_house_data_test_features.csv', index_col = 0
)

In [3]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [4]:
df.shape

(4322, 20)

In [5]:
import pickle

In [6]:
infile = open("data/model.pickle",'rb')
model = pickle.load(infile)
infile.close()

In [7]:
print(model.intercept_)
print(len(model.coef_))

-104.97630032186937
48


In [8]:
infile = open("data/other_info.pickle",'rb')
other_info = pickle.load(infile)
infile.close()

In [9]:
other_info

[0.0010755476417730245,
 0.0017952352048804102,
 7601.575840208134,
 Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
        'waterfront', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
        'sqft_living15', 'sqft_lot15', 'zip_mean_income', 'zip_total_families',
        'year_most_recent_construction', 'years_since_most_recent_construction',
        'yard', 'zip_family_ratio', 'zip_incomeXsqft_living',
        'waterfrontXsqft_living', 'zip_incomeXsqft_lot', 'sqft_living^2',
        'yearsince_mrcXcondition', 'yearsince_mrcXsqft_living',
        'waterfront_view', 'season_sold_Winter', 'condition_2', 'condition_5',
        'grade_cat_10', 'grade_cat_11', 'grade_cat_12', 'grade_cat_13',
        'grade_cat_6', 'grade_cat_7', 'grade_cat_8', 'grade_cat_9',
        'view_cat_good_view', 'view_cat_great_view', 'view_cat_ok_view',
        'excess_bathrooms_cat_1_less', 'excess_bathrooms_cat_1_or_more',
        'excess_bathrooms_cat_2_or_less', 'decade_built

In [10]:
bathroom_per_sqftl = other_info[0]
bedrooms_per_sqftl = other_info[1]
sqft_living_cap = other_info[2]
selected_columns = other_info[3]

## Feature Engineering for holdout set

In [11]:
df['bathrooms'] = np.where(df['bathrooms'] < .74, np.round(df['sqft_living'] * bathroom_per_sqftl),  df['bathrooms'])
df['bedrooms'] = np.where(df['bedrooms'] < 1, np.round(df['sqft_living'] * bedrooms_per_sqftl),  df['bedrooms'])
df['bedrooms'] = np.where(df['bedrooms'] > (bedrooms_per_sqftl*df['sqft_living']+5), np.round(df['sqft_living'] * bedrooms_per_sqftl),  df['bedrooms'])
df['sqft_living'] = np.where(df['sqft_living'] > sqft_living_cap,
                            sqft_living_cap,
                             df['sqft_living'])

In [12]:
zipcodes = pd.read_csv('data/ACSST5Y2019.S1901_data_with_overlays_2021-03-08T095143.csv', index_col = 0, header = 1)

In [13]:
zipcodes['zipcode'] = zipcodes['Geographic Area Name'].apply(lambda x: int(x[6:]))

In [14]:
zipcodes = zipcodes[['Estimate!!Households!!Total','Estimate!!Households!!Median income (dollars)', 'Estimate!!Households!!Mean income (dollars)', 'Estimate!!Families!!Total', 'zipcode']]

In [15]:
df = pd.merge(left = df, right =  zipcodes, on = 'zipcode', how = 'left')

In [16]:
df.rename(columns = {'Estimate!!Households!!Total': 'zip_total_households',
                     'Estimate!!Households!!Median income (dollars)': 'zip_median_income',
                   'Estimate!!Households!!Mean income (dollars)' : 'zip_mean_income',
                   'Estimate!!Families!!Total' : 'zip_total_families'},
                      inplace = True
                        )

In [17]:
df['zip_median_income'] = df['zip_median_income'].astype(int)
df['zip_mean_income'] = df['zip_mean_income'].astype(int)
df['zip_total_households'] = df['zip_total_households'].astype(int)
df['zip_total_families'] = df['zip_total_families'].astype(int)

In [18]:
df['grade_cat'] = np.where(df['grade'] < 6, 'low_grade', df['grade'])

In [19]:
conditions = [df['view'] == 0,
              df['view'] == 1,
              df['view'] == 2,
              df['view'] == 3,
              df['view'] == 4
             ]

choices = ['no_view',
           'ok_view',
           'ok_view',
           'good_view',
          'great_view']
           
    

df['view_cat'] = np.select(conditions, choices)

In [20]:
#year sold
df['year'] = df['date'].apply(lambda x: int(x[0:4]))

#month sold
df['month_sold'] = df['date'].apply(lambda x: int(x[4:6]))

#most recent year either year built or year renovated
df['year_most_recent_construction'] = np.where(df['yr_renovated'] > df['yr_built'], df['yr_renovated'], df['yr_built'])

#years since most recent construction
df['years_since_most_recent_construction'] = df['year'] - df['year_most_recent_construction']
#take out any negative values
df['years_since_most_recent_construction'] = np.where(df['years_since_most_recent_construction'] <0, 0, df['years_since_most_recent_construction'])

#yard size
df['yard'] = df['sqft_lot'] - (df['sqft_living']/df['floors'])

df['excess_bathrooms'] = np.ceil(df['bathrooms'] - df['bedrooms'])

#ratio of families to household per zipcode. Maybe zipcode where higher ratios of families live, will affect house prices
df['zip_family_ratio'] = df['zip_total_families'] / df['zip_total_households']

df['decade_built'] = df['yr_built'].apply(lambda x: str(x)[0:3]+'0s')

In [21]:
conditions = [df['excess_bathrooms'] <= -2,
              df['excess_bathrooms'] == -1,
              df['excess_bathrooms'] == 0,
              df['excess_bathrooms'] >= 1,

             ]

choices = ['2_or_less',
           '1_less',
           'parity',
           '1_or_more',]
           
    
df['excess_bathrooms_cat'] = np.select(conditions, choices)

In [22]:
conditions = [df['month_sold'] == 1,
              df['month_sold'] == 2,
              df['month_sold'] == 3,
              df['month_sold'] == 4,
              df['month_sold'] == 5,
              df['month_sold'] == 6,
              df['month_sold'] == 7,
              df['month_sold']  == 8,
              df['month_sold']  == 9,
              df['month_sold']  == 10,
              df['month_sold']  == 11,
              df['month_sold']  == 12
             ]

choices = ['Winter',
           'Winter',
           'Spring',
           'Spring',
           'Spring',
          'Summer',
          'Summer',
          'Summer',
          'Fall',
          'Fall',
          'Fall',
          'Winter']
           
    
df['season_sold'] = np.select(conditions, choices, default = np.NaN)

In [23]:
conditions = [df['decade_built'] == '1900s',
              df['decade_built'] == '1910s',
              df['decade_built'] == '1920s',
              df['decade_built'] == '1930s',
             ]

choices = ['Pre-war',
           'Pre-war',
           'Pre-war',
           'Pre-war']
           
    


df['decade_built_adjusted'] = np.select(conditions, choices, default = df['decade_built'])

In [24]:
df['zip_incomeXsqft_living'] = df['zip_mean_income'] * df['sqft_living']
df['waterfrontXsqft_living'] = df['waterfront'] * df['sqft_living']
df['zip_incomeXsqft_lot'] = df['zip_mean_income'] * df['sqft_lot']
df['sqft_living^2'] = df['sqft_living']**2
df['year_since_mrcXgrade'] = df['years_since_most_recent_construction'] * df['grade']
df['yearsince_mrcXcondition'] = df['years_since_most_recent_construction'] * df['condition']
df['yearsince_mrcXsqft_living'] = df['years_since_most_recent_construction']  * df['sqft_living']
df['waterfront_view'] = df['waterfront'] * df['view']

In [25]:
df_dummy = pd.get_dummies(df, columns=['season_sold', 'condition'], drop_first=True)
df_dummy = pd.get_dummies(df_dummy, columns = ['grade_cat']).drop(columns = 'grade_cat_low_grade', axis = 1)
df_dummy = pd.get_dummies(df_dummy, columns = ['view_cat']).drop(columns = 'view_cat_no_view', axis = 1)
df_dummy = pd.get_dummies(df_dummy, columns = ['excess_bathrooms_cat']).drop(columns = 'excess_bathrooms_cat_parity', axis = 1)
df = pd.get_dummies(df_dummy, columns = ['decade_built_adjusted']).drop(columns = 'decade_built_adjusted_2010s', axis = 1)

In [26]:
#dropping columns I preferred to categorize or don't needx
# df.drop(columns = ['decade_built', 'excess_bathrooms', 'zip_median_income'], inplace = True)

## Predict the holdout set

In [27]:
selected_columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       'sqft_living15', 'sqft_lot15', 'zip_mean_income', 'zip_total_families',
       'year_most_recent_construction', 'years_since_most_recent_construction',
       'yard', 'zip_family_ratio', 'zip_incomeXsqft_living',
       'waterfrontXsqft_living', 'zip_incomeXsqft_lot', 'sqft_living^2',
       'yearsince_mrcXcondition', 'yearsince_mrcXsqft_living',
       'waterfront_view', 'season_sold_Winter', 'condition_2', 'condition_5',
       'grade_cat_10', 'grade_cat_11', 'grade_cat_12', 'grade_cat_13',
       'grade_cat_6', 'grade_cat_7', 'grade_cat_8', 'grade_cat_9',
       'view_cat_good_view', 'view_cat_great_view', 'view_cat_ok_view',
       'excess_bathrooms_cat_1_less', 'excess_bathrooms_cat_1_or_more',
       'excess_bathrooms_cat_2_or_less', 'decade_built_adjusted_1940s',
       'decade_built_adjusted_1950s', 'decade_built_adjusted_19

In [28]:
len(selected_columns)

48

In [29]:
df[selected_columns]

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15,zip_mean_income,zip_total_families,year_most_recent_construction,years_since_most_recent_construction,yard,zip_family_ratio,zip_incomeXsqft_living,waterfrontXsqft_living,zip_incomeXsqft_lot,sqft_living^2,yearsince_mrcXcondition,yearsince_mrcXsqft_living,waterfront_view,season_sold_Winter,condition_2,condition_5,grade_cat_10,grade_cat_11,grade_cat_12,grade_cat_13,grade_cat_6,grade_cat_7,grade_cat_8,grade_cat_9,view_cat_good_view,view_cat_great_view,view_cat_ok_view,excess_bathrooms_cat_1_less,excess_bathrooms_cat_1_or_more,excess_bathrooms_cat_2_or_less,decade_built_adjusted_1940s,decade_built_adjusted_1950s,decade_built_adjusted_1960s,decade_built_adjusted_1990s,decade_built_adjusted_2000s,decade_built_adjusted_Pre-war
0,4.0,2.50,2270.0,11500,1.0,0,1540,730,1967,0,2020,10918,128063,11334,1967,47,9230.0,0.642299,290703010.0,0.0,1472724500,5152900.0,141,106690.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
1,4.0,2.50,2270.0,11500,1.0,0,1540,730,1967,0,2020,10918,128063,11334,1967,48,9230.0,0.642299,290703010.0,0.0,1472724500,5152900.0,144,108960.0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
2,3.0,2.50,1470.0,1779,2.0,0,1160,310,2005,0,1470,1576,155934,8246,2005,9,1044.0,0.731612,229222980.0,0.0,277406586,2160900.0,27,13230.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,3.0,1.75,1280.0,16200,1.0,0,1030,250,1976,0,1160,10565,221823,4041,1976,38,14920.0,0.885020,283933440.0,0.0,3593532600,1638400.0,114,48640.0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,4.0,2.75,2830.0,8126,2.0,0,2830,0,2005,0,2830,7916,137426,10323,2005,10,6711.0,0.748315,388915580.0,0.0,1116723676,8008900.0,30,28300.0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,3.0,2.50,1530.0,1131,3.0,0,1530,0,2009,0,1530,1509,133001,10600,2009,5,621.0,0.424102,203491530.0,0.0,150424131,2340900.0,15,7650.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4318,4.0,2.50,2310.0,5813,2.0,0,2310,0,2014,0,1830,7200,100923,6887,2014,1,4658.0,0.675196,233132130.0,0.0,586665399,5336100.0,3,2310.0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4319,2.0,0.75,1020.0,1350,2.0,0,1020,0,2009,0,1020,2007,120821,7206,2009,5,840.0,0.516670,123237420.0,0.0,163108350,1040400.0,15,5100.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
4320,3.0,2.50,1600.0,2388,2.0,0,1600,0,2004,0,1410,1287,141468,8463,2004,11,1588.0,0.708497,226348800.0,0.0,337825584,2560000.0,33,17600.0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [30]:
final_answers = np.exp(model.predict(df[selected_columns]))

## Export Predictions

In [31]:
final = pd.DataFrame(final_answers)

In [32]:
#sanity check
final.describe()

,0
count,4.322000e+03
mean,5.549860e+05
std,3.244696e+05
min,1.382192e+05
25%,3.500140e+05
50%,4.722336e+05
75%,6.454914e+05
max,3.134260e+06


In [33]:
final.to_csv('data/housing_preds_gary_schwaeber.csv')